# Monte Carlo Variance Reduction

Objective of reduce the variance of results from a Monte Carlo Simulation when valuing derivatives, references: Implementing Derivatives Models

# 1.0 Import Libraries

In [49]:
import numpy as np
import pandas as pd
import math
import datetime
import scipy.stats as stats
import matplotlib.pyplot as plt

# 2.0 Set Parameters

Let´s Write 2 options contracts in differents assets, thais is perfectly negatively correlated ( VIX & SP500 as example), imagine that the 2 assets have the same price at the moment 

In [50]:
# parameters
S=101.15
K=98.01
vol=0.0991
r=0.015
N=10 # time steps
M=1000 # simulations

value=3.86 # Market Price of Option

T=((datetime.date(2022,3,17)-datetime.date(2022,1,17)).days+1)/365 # Days to Expiration
h=T
print(T)

0.1643835616438356


# 3.0 Solution

In [53]:

N=1
dt=T/N
nudt=(r-0.5*vol**2)*dt
volsdt=vol*np.sqrt(dt)
lnS=np.log(S)
#Monte Carlo:
Z=np.random.normal(size=(N,M))
delta_lnSt1=nudt+volsdt*Z
delta_lnSt2=nudt-volsdt*Z
lnSt1=lnS+np.cumsum(delta_lnSt1,axis=0)
lnSt2=lnS+np.cumsum(delta_lnSt2,axis=0)
# expectations
ST1=np.exp(lnSt1)
ST2=np.exp(lnSt2)
CT=0.5*(np.maximum(0,ST1[-1]-K)+np.maximum(0,ST2[-1]-K))
C0=np.exp(-r*T)*np.sum(CT)/M
sigma=np.sqrt(np.sum((CT-C0)**2)/(M-1))
SE=sigma/math.sqrt(M)
print(f"Price of Call: {C0:.2f} and Error +- {SE:.2f}")

Price of Call: 3.83 and Error +- 0.03


# Simple Black & Scholes

In [54]:
from sympy import *
import sympy as sp
x, y ,z, S, q, T, r,K,v =sp.symbols("x y z S q T r K v ")
(x,y,z,S,q,T,r,K,v)


(x, y, z, S, q, T, r, K, v)

In [55]:
def CDF(value):
    CDF = integrate((1/sqrt(2*pi*1))*exp(-1*((x-0)**2)/(2*1**2)),(x,-oo,value))
    return(N(CDF))
d1=(ln(S/K)+(r-q+0.5*v**2)*T)/(v*sqrt(T))
d2=d1-v*sqrt(T)
black_scholes_call= S*exp(-q*T)*CDF(d1)-exp(-r*T)*K*CDF(d2)
black_scholes_put= exp(-r*T)*K*CDF(-d2) -S*exp(-q*T) *CDF(-d1)
black_scholes_call 

-K*(0.5*erf(sqrt(2)*(-sqrt(T)*v + (T*(-q + r + 0.5*v**2) + log(S/K))/(sqrt(T)*v))/2) + 0.5)*exp(-T*r) + S*(0.5*erf(sqrt(2)*(T*(-q + r + 0.5*v**2) + log(S/K))/(2*sqrt(T)*v)) + 0.5)*exp(-T*q)

In [56]:
print(N(black_scholes_call.subs(K,98.01).subs(S,101.15).subs(v,0.0991).subs(T,h).subs(q,0).subs(r,0.015)))

3.82371240251315


The prices aproximate...